
# Question 1: Given some sample data, write a program to answer the following

On Shopify, we have exactly 100 sneaker shops, and each of these shops sells only one model of shoe. We want to do some analysis of the average order value (AOV). When we look at orders data over a 30 day window, we naively calculate an AOV of $3145.13. Given that we know these shops are selling sneakers, a relatively affordable item, something seems wrong with our analysis. 

* Think about what could be going wrong with our calculation. Think about a better way to evaluate this data. 
* What metric would you report for this dataset?
* What is its value?

In [21]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [64]:
df = pd.read_excel('2019 Winter Data Science Intern Challenge Data Set.xlsx', engine="openpyxl")

In [65]:
df.head()

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
0,1,53,746,224,2,cash,2017-03-13 12:36:56.190022
1,2,92,925,90,1,cash,2017-03-03 17:38:51.999116
2,3,44,861,144,1,cash,2017-03-14 04:23:55.594730
3,4,18,935,156,1,credit_card,2017-03-26 12:43:36.648760
4,5,18,883,156,1,credit_card,2017-03-01 04:35:10.772536


In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   order_id        5000 non-null   int64         
 1   shop_id         5000 non-null   int64         
 2   user_id         5000 non-null   int64         
 3   order_amount    5000 non-null   int64         
 4   total_items     5000 non-null   int64         
 5   payment_method  5000 non-null   object        
 6   created_at      5000 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(5), object(1)
memory usage: 273.6+ KB


In [67]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
order_id,5000.0,2500.5000,1443.520003,1.0,1250.75,2500.5,3750.25,5000.0
shop_id,5000.0,50.0788,29.006118,1.0,24.00,50.0,75.00,100.0
user_id,5000.0,849.0924,87.798982,607.0,775.00,849.0,925.00,999.0
order_amount,5000.0,3145.1280,41282.539349,90.0,163.00,284.0,390.00,704000.0
total_items,5000.0,8.7872,116.320320,1.0,1.00,2.0,3.00,2000.0


## ***Part 1: Think about what could be going wrong with our calculation. Think about a better way to evaluate this data.***

Average order value (AOV) is defined as the ratio of total revenue accrued to the total orders placed over a given period of time &rarr; $\frac{\Sigma_t\text{revenue}}{\Sigma_t\text{orders}} = \text{AOV}$. 

From [bigcommerce.com](https://www.bigcommerce.com/ecommerce-answers/what-average-order-value/):

"*AOV is determined using sales per order, not sales per customer. Although one customer may come back multiple times to make a purchase, each order would be factored into AOV separately.*"

In [68]:
# Since Order ID is a unique identifier we can use this to represent each order with a labeled index

df = df.set_index("order_id")

In [70]:
df.head()

,shop_id,user_id,order_amount,total_items,payment_method,created_at
order_id,,,,,,
1,53,746,224,2,cash,2017-03-13 12:36:56.190022
2,92,925,90,1,cash,2017-03-03 17:38:51.999116
3,44,861,144,1,cash,2017-03-14 04:23:55.594730
4,18,935,156,1,credit_card,2017-03-26 12:43:36.648760
5,18,883,156,1,credit_card,2017-03-01 04:35:10.772536


In [93]:
df.nunique() # There are 5000 orders but only 4995 unique time stamps? Let's investigate

shop_id            100
user_id            301
order_amount       258
total_items          8
payment_method       3
created_at        4995
dtype: int64

In [105]:
df["created_at"].value_counts().sort_values(ascending=False) # We have 4 entries with duplicate time stamps

2017-03-28 04:00:00.000000    3
2017-03-24 04:00:00.000000    2
2017-03-07 04:00:00.000000    2
2017-03-02 04:00:00.000000    2
2017-03-09 21:53:32.040932    1
                             ..
2017-03-16 01:14:41.639466    1
2017-03-09 13:19:49.144626    1
2017-03-02 13:15:36.411109    1
2017-03-17 16:38:06.279001    1
2017-03-02 08:17:10.568191    1
Name: created_at, Length: 4995, dtype: int64

In [81]:
ser = df["created_at"].value_counts()
dup_ts = ser[ser > 1].index

In [82]:
df[df["created_at"].isin(dupe_ts)]

,shop_id,user_id,order_amount,total_items,payment_method,created_at
order_id,,,,,,
16,42,607,704000,2000,credit_card,2017-03-07 04:00:00
521,42,607,704000,2000,credit_card,2017-03-02 04:00:00
1105,42,607,704000,2000,credit_card,2017-03-24 04:00:00
2298,42,607,704000,2000,credit_card,2017-03-07 04:00:00
2836,42,607,704000,2000,credit_card,2017-03-28 04:00:00
2970,42,607,704000,2000,credit_card,2017-03-28 04:00:00
3333,42,607,704000,2000,credit_card,2017-03-24 04:00:00
4057,42,607,704000,2000,credit_card,2017-03-28 04:00:00
4647,42,607,704000,2000,credit_card,2017-03-02 04:00:00


It appears that these transactions may have been recorded multiple times as their entire entries are equivalent down to the second. These duplicate order entries are more than likely contributing to the inflated AOV. We can remove these entries and then recalculate AOV to see if we get a more accurate result.

In [83]:
dup_ts_idxs = df[df["created_at"].isin(dupe_ts)].drop([16, 521, 1105, 2836], axis=0).index

In [109]:
# There are 5 duplicates so we should have 5 fewer entries after dropping

len(df.drop(dup_ts_idxs, axis=0)), len(df)

(4995, 5000)

In [85]:
dropped_tsdupes = df.drop(dup_ts_idxs, axis=0)

In [92]:
revenue = dropped_tsdupes["order_amount"].sum() 
orders = len(dropped_tsdupes)

recalculated_AOV = round(revenue/orders, 2)
recalculated_AOV

2443.57

Our recalculated_AOV still appears a bit high. This is not too surprising, however, given the spread of our data. The minimum order amount is $\$90$, the maximum order amount $\$704,000$ and the standard deviation of order amount is $\approx\$41,282.54$. Notice that our first and third quartiles are in a reasonable expenditure range, implying that the top $\%25$ of data contains extreme outliers. These outliers are more than likely causing an inflated AOV so we should utilize a different metric in order to fully understand our business performance.

## ***Part 2: What metric would you report for this dataset?***

Due to my my lack of domain knowledge I am unsure of the KPI's we can procure with the given columns of data, however I decided to define a metric that might be pertinent in an eCommerce business. As a business owner I may be interested in increasing the number of transactions per customer. Perhaps we could look at every person who purchased only 1 item and encourage them to buy multiple items through bundles or free shipping. We will define this metric as Single Purchase Percentage (SPP).

## ***Part 3: What is its value?***

### Calculating SPP
We can find the SPP by using pandas' .value_counts() method for the total_items column and divide the single purchase entry by the sum of the remaining purchase entries.

In [121]:
purchase_counts = dropped_tsdupes["total_items"].value_counts()
purchase_counts

2       1832
1       1830
3        941
4        293
5         77
2000      12
6          9
8          1
Name: total_items, dtype: int64

In [122]:
single_purchases = purchase_counts[1]
single_purchases

1830

In [123]:
spp = round(100*(single_purchases / purchase_counts.sum()), 2)
spp

36.64

__Projected SPP: $\%36.64$__. Given that the number of orders either increases or remains constant, a decrease in SPP would imply that people are purchasing more than one item per order. 